In [ ]:
import datetime
import lxml.html
import MySQLdb as mdb

from math import ceil

In [13]:
def obtain_parse_wiki_snp500():
  
    """ 
  Download and parse the Wikipedia list of S&P500 
  constituents using requests and libxml.

  Returns a list of tuples for to add to MySQL.
    """

    # Stores the current time, for the created_at record
    now = datetime.datetime.utcnow()
    # Use libxml to download the list of S&P500 companies and obtain the symbol table
    page = lxml.html.parse('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    symbolslist = page.xpath('//table[1]/tr')[1:]
    
    # Obtain the symbol information for each row in the S&P500 constituent table
    symbols = []

    for symbol in symbolslist:
        tds = symbol.getchildren()
        sd = {'ticker': tds[0].getchildren()[0].text,
        'name': tds[1].getchildren()[0].text,
        'sector': tds[3].text}

    # Create a tuple (for the DB format) and append to the grand list
    symbols.append( (sd['ticker'], 'stock', sd['name'], sd['sector'], 'USD', now, now) )
    return symbols
    

In [ ]:
def insert_snp500_symbols(symbols):
  """
  Insert the S&P500 symbols into the MySQL database.
  """

    # Connect to the MySQL instance
    db_host = 'localhost'
    db_user = 'sec_user'
    db_pass = 'password'
    db_name = 'securities_master'
    con = mdb.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)
    
    # Create the insert strings
    column_str = "ticker, instrument, name, sector, currency, created_date, last_updated_date"
    insert_str = ("%s, " * 7)[:-2]
    final_str = "INSERT INTO symbol (%s) VALUES (%s)" % (column_str, insert_str)
    print final_str, len(symbols)

    # Using the MySQL connection, carry out an INSERT INTO for every symbol
    with con: 
        cur = con.cursor()
        # This line avoids the MySQL MAX_PACKET_SIZE
        # Although of course it could be set larger!
    
        for i in range(0, int(ceil(len(symbols) / 100.0))):
            cur.executemany(final_str, symbols[i*100:(i+1)*100-1])

In [ ]:

if __name__ == "__main__":
    
    symbols = obtain_parse_wiki_snp500()
    insert_snp500_symbols(symbols)

In [ ]:
import datetime
import MySQLdb as mdb
import urllib2

In [ ]:
# Obtain a database connection to the MySQL instance

db_host = 'localhost'
db_user = 'sec_user'
db_pass = 'password'
db_name = 'securities_master'
con = mdb.connect(db_host, db_user, db_pass, db_name)

In [ ]:
def obtain_list_of_db_tickers():
  
    # Obtains a list of the ticker symbols in the database.
    with con: 
        cur = con.cursor()
        cur.execute("SELECT id, ticker FROM symbol")
        data = cur.fetchall()
        
        return [(d[0], d[1]) for d in data]
    

In [17]:
def get_daily_historic_data_yahoo(ticker,
                      start_date=(2000,1,1),
                      end_date=datetime.date.today().timetuple()[0:3]):

    """ 
    Obtains data from Yahoo Finance returns and a list of tuples.
    ticker: Yahoo Finance ticker symbol, e.g. "GOOG" for Google, Inc.
    start_date: Start date in (YYYY, M, D) format
    end_date: End date in (YYYY, M, D) format
    """
    
    # Construct the Yahoo URL with the correct integer query parameters
    # for start and end dates. Note that some parameters are zero-based!

    yahoo_url = "http://ichart.finance.yahoo.com/table.csv?s=%s&a=%s&b=%s&c=%s&d=%s&e=%s&f=%s" % \
      (ticker, start_date[1] - 1, start_date[2], start_date[0], end_date[1] - 1, end_date[2], end_date[0])
    
    
    # Try connecting to Yahoo Finance and obtaining the data
    # On failure, print an error message.
    try:
        yf_data = urllib.urlopen(yahoo_url).readlines()[1:] # Ignore the header
        prices = []
        for y in yf_data:
            p = y.strip().split(',')
            prices.append( (datetime.datetime.strptime(p[0], '%Y-%m-%d'),
                            p[1], p[2], p[3], p[4], p[5], p[6]) )
            
    except Exception, e:
        print ("Could not download Yahoo data: %s" % e)
        
    return prices

In [18]:
def insert_daily_data_into_db(data_vendor_id, symbol_id, daily_data):
  """
  Takes a list of tuples of daily data and adds it to the
  MySQL database. Appends the vendor ID and symbol ID to the data.

  daily_data: List of tuples of the OHLC data (with 
  adj_close and volume)
  """

    # Create the time now
    now = datetime.datetime.utcnow()
    
    # Amend the data to include the vendor ID and symbol ID
    daily_data = [(data_vendor_id, symbol_id, d[0], now, now,
               d[1], d[2], d[3], d[4], d[5], d[6]) for d in daily_data]
    
    # Create the insert strings
    column_str = """data_vendor_id, symbol_id, price_date, created_date, 
             last_updated_date, open_price, high_price, low_price, 
             close_price, volume, adj_close_price"""
    insert_str = ("%s, " * 11)[:-2]

    final_str = "INSERT INTO daily_price (%s) VALUES (%s)" % (column_str, insert_str)
    
    # Using the MySQL connection, carry out an INSERT INTO for every symbol
    with con:
        cur = con.cursor()
        cur.executemany(final_str, daily_data)

In [ ]:
if __name__ == "__main__":
  # Loop over the tickers and insert the daily historical
  # data into the database
    
    tickers = obtain_list_of_db_tickers()
    for t in tickers:
        print ("Adding data for %s" % t[1])

        yf_data = get_daily_historic_data_yahoo(t[1])
        insert_daily_data_into_db('1', t[0], yf_data)
        
        try:
            yf_data = urllib.urlopen(yahoo_url).readlines()
        except Exception, e:
            print ("Could not download Yahoo data: %s" % e)
    

In [ ]:
# Create the (temporary) Python data structures to store the historical data

date_list = []
hist_data = [[] for i in range(6)]

In [ ]:
# Format and copy the raw text data into datetime objects
# and floating point values (still in native Python lists)

for day in yf_data[1:]:  # Avoid the header line in the CSV
    headers = day.rstrip().split(',')
    date_list.append(datetime.datetime.strptime(headers[0],'%Y-%m-%d'))
    for i, header in enumerate(headers[1:]):
        hist_data[i].append(float(header))
    

In [ ]:
# Create a Python dictionary of the lists and then use that to
# form a sorted Pandas DataFrame of the historical data

hist_data = dict(zip(['open', 'high', 'low', 'close', 'volume', 'adj_close'], hist_data))
pdf = pd.DataFrame(hist_data, index=pd.Index(date_list)).sort()

return pdf

In [ ]:
def annualised_sharpe(returns, N=252):

    """
    Calculate the annualised Sharpe ratio of a returns stream 
    based on a number of trading periods, N. N defaults to 252,
    which then assumes a stream of daily returns.

    The function assumes that the returns are the excess of 
    those compared to a benchmark.
    """
    
    return np.sqrt(N) * returns.mean() / returns.std()


In [ ]:
def equity_sharpe(ticker):
    
    """
    Calculates the annualised Sharpe ratio based on the daily
    returns of an equity ticker symbol listed in Yahoo Finance.

    The dates have been hardcoded here for the QuantStart article 
    on Sharpe ratios.
    """

    # Obtain the equities daily historic data for the desired time period
    # and add to a pandas DataFrame
    pdf = get_historic_data(ticker, start_date=(2000,1,1), end_date=(2013,5,29))

    # Use the percentage change method to easily calculate daily returns
    pdf['daily_ret'] = pdf['adj_close'].pct_change()

    # Assume an average annual risk-free rate over the period of 5%
    pdf['excess_daily_ret'] = pdf['daily_ret'] - 0.05/252

    # Return the annualised Sharpe ratio based on the excess daily returns
    return annualised_sharpe(pdf['excess_daily_ret'])

In [ ]:
def market_neutral_sharpe(ticker, benchmark):
    
    """
    Calculates the annualised Sharpe ratio of a market
    neutral long/short strategy inolving the long of 'ticker'
    with a corresponding short of the 'benchmark'.
    """

    # Get historic data for both a symbol/ticker and a benchmark ticker
    # The dates have been hardcoded, but you can modify them as you see fit!
    tick = get_historic_data(ticker, start_date=(2000,1,1), end_date=(2013,5,29))
    bench = get_historic_data(benchmark, start_date=(2000,1,1), end_date=(2013,5,29))
    
    # Calculate the percentage returns on each of the time series
    tick['daily_ret'] = tick['adj_close'].pct_change()
    bench['daily_ret'] = bench['adj_close'].pct_change()
    
    # Create a new DataFrame to store the strategy information
    # The net returns are (long - short)/2, since there is twice 
    # trading capital for this strategy
    strat = pd.DataFrame(index=tick.index)
    strat['net_ret'] = (tick['daily_ret'] - bench['daily_ret'])/2.0
    
    # Return the annualised Sharpe ratio for this strategy
    return annualised_sharpe(strat['net_ret'])
